# LFP example


In [1]:
import altair as alt
from bayes_window.generative_models import *
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import BayesWindow, models

## Make and visualize model oscillation power
40 trials of "theta power" is generated for every animal. It is drawn randomly as a poisson process.

This is repeated for "stimulation" trials, but poisson rate is higher.

In [2]:
# Draw some fake data:
df, df_monster, index_cols, _ = generate_fake_lfp(mouse_response_slope=9, n_trials=30)

Mice vary in their baseline power.

Higher-baseline mice tend to have smaller stim response:

In [3]:
c1=plot_data(df=df,x='stim',y='Log power').properties(width=60)
c2=plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

In [4]:
plot_data(df=df,x='stim',y='Log power',color='mouse').properties(width=80)

alt.LayerChart(...)

## Fit a Bayesian hierarchical model and plot slopes
In a hierarchical model, parameters are viewed as a sample from a population distribution of parameters. Thus, we view them as being neither entirely different or exactly the same. This is ***partial pooling***:

![hierarchical](../motivation/parpooled.png)
This model allows intercepts to vary across mouse, according to a random effect. We just add a fixed slope for the predictor (i.e all mice will have the same slope):

$$y_i = \alpha_{j[i]} + \beta x_{i} + \epsilon_i$$

where:
- $j$ is mouse index
- $i$ is observation index
- $y_i$ is observed power
- $x_i$ is 0 (no stimulation) or 1 (stimulation)
- $\epsilon_i \sim N(0, \sigma_y^2)$, error
- $\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$, Random intercept

We set a separate intercept for each mouse, but rather than fitting separate regression models for each mouse, multilevel modeling **shares strength** among mice, allowing for more reasonable inference in mice with little data.

The wrappers in this library allow us to fit and plot this inference in just three lines of code. Under the hood, it uses the following Numpyro code:
```python
# Given: y, treatment, group, n_subjects
# Sample intercepts
a = sample('a', Normal(0, 1))
a_subject = sample('a_subject', Normal(jnp.tile(0, n_subjects), 1))

# Sample variances
sigma_a_subject = sample('sigma_a', HalfNormal(1))
sigma_obs = sample('sigma_obs', HalfNormal(1))

# Sample slope - this is what we are interested in!
b = sample('b_stim', Normal(0, 1))

# Regression equation
slope = a + a_subject[group] * sigma_a_subject + b * treatment

# Sample power
sample('y', Normal(theta, sigma_obs), obs=y)
```


Above is the contents of `model_hier_stim_one_codition.py`, the function passed as argument in line 4 below.

In [5]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
# Fit:
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='student');

# Plot:
chart_power_difference = window.plot(independent_axes=False,
                                    add_posterior_density=True,add_box=False
                                    ).properties(title='Posterior')

chart_power_difference

alt.LayerChart(...)

In this chart:

- The blue dot is the mean of posterior

- The black line is the 94% highest density interval

- The boxplot is made from difference between groups in the data (no fitting)


## Compare to non-bayesian approaches

ANOVA does not pick up the effect of stim as significant:

In [6]:
window.fit_anova();

Log_power~stim
             sum_sq    df         F    PR(>F)
stim      0.068460   1.0  3.123298  0.098961
Residual  0.306868  14.0       NaN       NaN


A linear mixed-effect model shows the effect of stim (slope) as significant. It includes random intercepts of mouse:

In [8]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
window.fit_lme(add_data=False);

Using formula Log_power ~ (1 | mouse) + (stim | mouse)


LinAlgError: Singular matrix

In [ ]:
chart_power_difference_lme=window.plot_posteriors_slopes().properties(title='LME')
chart_power_difference_lme

## Compare LME and Bayesian slopes side by side

In [ ]:
chart_power_difference|chart_power_difference_lme

## Inspect Bayesian result further
We named intercept _a\_subject_, so now we can plot it for each animal

In [ ]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
# Fit slopes again:
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='normal');
intercepts=window.trace.posterior['a_subject'].mean(['chain','draw']).to_dataframe().reset_index()
# Plot
chart_intercepts=alt.Chart(intercepts).mark_bar().encode(
    x=alt.X('mouse:O',title='Mouse'),
    y=alt.Y('a_subject', title='Intercept')
)
chart_intercepts

Our plotting backend's flexibility allows us to easily concatenate multiple charts in the same figures with the | operator:

In [ ]:
chart_intercepts | chart_power_difference | chart_power_difference_lme

## Include all samples in each trial
The mean of every one of the 30 trials we drew for each mouse is a manifestation of the same underlying process that generates power for each mouse. Let's try to include all samples that come in each trial

In [ ]:
# Initialize:
window=BayesWindow(df_monster,#.groupby(['mouse','stim', ]).sample(n=230),
                   y='Log power', treatment='stim', group='mouse')
# Fit:
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,num_warmup=1000,
                  do_make_change='subtract', dist_y='normal', progress_bar=True);

# Plot:
chart_power_difference_monster=window.plot(independent_axes=False).properties(title='Posterior')
chart_power_difference_monster

Much tighter credible intervals here!

Same with linear mixed model:

In [ ]:
alt.data_transformers.disable_max_rows()
window=BayesWindow(df_monster,
                   y='Log power', treatment='stim', group='mouse')
window.fit_lme()


chart_power_difference_monster_lme=window.plot_posteriors_slopes().properties(title='LME')
chart_power_difference_monster_lme

In [ ]:
chart_power_difference_monster | chart_power_difference_monster_lme